In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Visualización 2 - bokeh y plotly

## Dataset

En esta clase vamos a usar datos de COVID-19 que disponibiliza Johns Hopkins University en 

https://github.com/CSSEGISandData/COVID-19

Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE 

(CSSE = Computer Science and Software Engineering)

Los datasets que usamos se actualizan diariamente y llevan el registro de cantidad de confirmados, muertos y recuperados por país o región.

Usando estos datos vamos a construir algunas visualizaciones con las bibliotecas bokeh y plotly.

(Nota: la actualizacion es diaria en mayo 2020.)


## Imports

In [ ]:
import pandas as pd
import numpy as np
import datetime

### Leemos los datos

In [ ]:
world_confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
world_deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
world_recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

world_confirmed = pd.read_csv(world_confirmed_url)
world_deaths = pd.read_csv(world_deaths_url)
world_recovered = pd.read_csv(world_recovered_url)

In [ ]:
world_confirmed.head(3)

In [ ]:
world_deaths.head(3)

In [ ]:
world_recovered.head(3)

### Ejercicio 1

Veamos de qué paises tiene datos este dataset.

Filtremos los datos de países latinoamericanos y construyamos tres nuevos DataFrame 

* world_confirmed_latam
* world_deaths_latam
* world_recovered_latam

In [ ]:
## [BORRAR_PRESENCIAL]
world_confirmed["Country/Region"].unique()

Empezamos construyendo una lista con los países de interes:

In [ ]:
latam_countries = ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia',
       'Costa Rica', 'Cuba', 'Dominican Republic', 'Ecuador',
       'El Salvador', 'Guatemala', 'Honduras',
       'Mexico', 'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela', 'Belize']

In [ ]:
## [BORRAR_PRESENCIAL]

world_confirmed_latam_mask = [x in latam_countries for x in world_confirmed["Country/Region"]]
world_confirmed_latam = world_confirmed.loc[world_confirmed_latam_mask, :]
world_confirmed_latam.head(3)

In [ ]:
## [BORRAR_PRESENCIAL]

world_deaths_latam_mask = [x in latam_countries for x in world_deaths["Country/Region"]]
world_deaths_latam = world_deaths.loc[world_deaths_latam_mask, :]
world_deaths_latam.head(3)


In [ ]:
## [BORRAR_PRESENCIAL]

world_recovered_latam_mask =  [x in latam_countries for x in world_recovered["Country/Region"]]
world_recovered_latam = world_recovered.loc[world_recovered_latam_mask, :]
world_recovered_latam.head(3)


Observemos que en todos estos DataFrame la fecha aparece como una columna, es decir a medida que pasa el tiempo crece la cantidad de columnas, y no la cantidad de filas (como es usual).



---

## Bokeh

### Imports

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20
from bokeh.models import HoverTool


### Ejercicio 2

Grafiquemos las muertes por día para los países de latinoamérica

Cada punto de este gráfico tendrá como coordena x la cantidad de días desde el 22 de enero del 2020, y como coordenada y la cantidad de muertes ese día. Cada país estará representado por una serie distinta.

**2.a** Para eso tenemos que construir un DataFrame con tres columnnas:
* coord_x: es la cantidad de días desde el 22 de enero del 2020 (fecha mínima en el dataset)
* coord_y: es la cantidad de muertes en la fecha y país correspondiente a ese registro
* color: es el país

Ayuda: usemos la función `melt` de pandas

https://pandas.pydata.org/docs/reference/api/pandas.melt.html

**2.b** Grafiquemos los datos del DataFrame que construimos en el punto anterior

In [ ]:
# vemos cuáles son las columnas en el DataFrame
print(world_deaths_latam.columns)

# armamos una lista con las columnas que queremos exluir
exclude_columns = ['Province/State', 'Country/Region', 'Lat', 'Long']

id_vars = ['Country/Region']
value_vars = [element for element in world_deaths_latam.columns if element not in exclude_columns]
world_deaths_latam_plot = pd.melt(world_deaths_latam, id_vars=id_vars, value_vars=value_vars)
print(world_deaths_latam_plot.head(3))

world_deaths_latam_plot.head(3)

Veamos si las columnas del DataFrame construido tiene el tipo de datos adecuado.

In [ ]:
world_deaths_latam_plot.dtypes

Para representar la fecha (columna "variable") como coordenada x de los puntos vamos a convertirlas en enteros: cantidad de días desde el 22 de enero de 2020 hasta la fecha.

In [ ]:
umbral_timestamp = pd.Timestamp(day = 22, month = 1, year = 2020)

variable_days = world_deaths_latam_plot.variable.apply(lambda x: (pd.Timestamp(x) - umbral_timestamp).days)

world_deaths_latam_plot["variable_days"] = variable_days
#world_deaths_latam_plot.head(3)

Vamos a usar para representar cada país un color de la paleta `Category20`.

Para acceder a un color de esa paleta necesitamos un dato entero entre 0 y 19.

Veamos cómo funciona.

`Category20[20]` tiene 20 colores. 

Si asociamos cada país con un entero (i), podemos representar ese país con el valor `Category20[20][i]`

In [ ]:
Category20[20]

Creamos un diccionario `color_map` que mapee países con colores 

In [ ]:
keys = world_deaths_latam_plot["Country/Region"].unique()

colors = [Category20[20][i] for i in range(20) ]
colormap = dict(zip(keys, colors))
colormap

In [ ]:
world_deaths_latam_plot = world_deaths_latam_plot.rename({
    'Country/Region': 'country',
    'variable_days': 'coord_x',
    'value': 'coord_y'
}, axis=1)

In [ ]:
world_deaths_latam_plot

Ahora podemos graficar los datos del DataFrame que construimos


In [ ]:

#Specify the selection tools to be made available
select_tools = ['box_zoom', 'pan', 'wheel_zoom', 'reset', 'crosshair', 'save']

# Format the tooltip
tooltips = [
    ('', '@country'),
    ("(x,y)", "($x, $y)")
]

# Create a blank figure with labels
p = figure(plot_width = 800, plot_height = 600, 
           title = 'Deaths in Latam',
           x_axis_label = 'Date (timestamp)', 
           y_axis_label = 'Deaths count',
           tools=select_tools)

## en el checkpoint dibujar solo dos series

for country in colormap.keys():
    country_mask =  world_deaths_latam_plot['country'] == country
    country_data = world_deaths_latam_plot.loc[country_mask]
    # Add circle glyph
    p.circle('coord_x', 'coord_y', size = 6,
        color = colormap[country], 
        legend_label = country, source = country_data)

p.legend.location = "top_left"
p.legend.click_policy='hide'

p.add_tools(HoverTool(tooltips=tooltips))


# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)


### Ejercicio 3

Repitamos el gráfico del ejercicio anterior, usando como valores en el eje x la cantidad de días desde la primera muerte en un país de latinoamérica.

Y usemos lineas para unir los puntos de la misma serie.

Ayuda: Calculemos el mínimo valor de días con cantidad de muertes distinto de cero.

In [ ]:
deaths_gt0_mask = world_deaths_latam_plot.coord_y > 0
deaths_gt0 = world_deaths_latam_plot.loc[deaths_gt0_mask, :]
x_umbral = deaths_gt0.coord_x.min()
print(x_umbral)

data_ej3_mask =  world_deaths_latam_plot.coord_x >= x_umbral
data_ej3 = world_deaths_latam_plot.loc[data_ej3_mask, :]
data_ej3

In [ ]:
# veamos que data_ej3 sigue teniendo registros con coord_y = 0 
# porque todos los países comenzaron a tener muertos en fechas diferentes y nosotros consideramos umbral la fecha del primer muerto
(data_ej3.coord_y == 0).sum()

Grafiquemos los datos de data_ej3

In [ ]:

#Specify the selection tools to be made available
select_tools = ['box_zoom', 'pan', 'wheel_zoom', 'reset', 'crosshair', 'save']

# Format the tooltip
tooltips = [
    ('', '@country'),
    ("(x,y)", "($x, $y)")
]

# Create a blank figure with labels
p = figure(plot_width = 800, plot_height = 600, 
           title = 'Deaths in Latam',
           x_axis_label = 'Date (timestamp)', 
           y_axis_label = 'Deaths count',
           tools=select_tools)

## en el checkpoint dibujar solo dos series

for country in colormap.keys():
    country_mask =  data_ej3['country'] == country    
    country_data = data_ej3.loc[country_mask]
    
    # Add line glyph
    p.line('coord_x', 'coord_y', line_width = 2,
           color = colormap[country], alpha=0.8, 
           legend_label = country, source = country_data)
    
    # Add circle glyph
    p.circle('coord_x', 'coord_y', size = 6,
        color = colormap[country], 
        legend_label = country, source = country_data)
    
    
p.legend.location = "top_left"
p.legend.click_policy='hide'

p.add_tools(HoverTool(tooltips=tooltips))


# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

### Ejercicio 4

Para poder comparar las curvas de distintos países, grafiquemos ahora en el eje x la cantidad de días desde las primeras tres muertes en cada país.

Para eso vamos a tener que calcular las coordenadas del eje x con este criterio.

In [ ]:
world_deaths_latam_plot["variable_timestamp"] = world_deaths_latam_plot.variable.apply(lambda x: pd.Timestamp(x))
world_deaths_latam_plot_deathgt3_mask = world_deaths_latam_plot.coord_y >= 3

data_ej4 = world_deaths_latam_plot.loc[world_deaths_latam_plot_deathgt3_mask, :]
fecha_tres_muertos = data_ej4.groupby("country")["variable_timestamp"].min()
fecha_tres_muertos

In [ ]:
world_deaths_latam_plot["fecha_tres_muertos"] = world_deaths_latam_plot.country.map(fecha_tres_muertos)

In [ ]:
world_deaths_latam_plot["delta"] = (world_deaths_latam_plot.variable_timestamp - world_deaths_latam_plot.fecha_tres_muertos)\
    .apply(lambda x: x.days)

In [ ]:
data_ej4 = world_deaths_latam_plot.loc[world_deaths_latam_plot_deathgt3_mask, :]
data_ej4

In [ ]:

#Specify the selection tools to be made available
select_tools = ['box_zoom', 'pan', 'wheel_zoom', 'reset', 'crosshair', 'save']

# Format the tooltip
tooltips = [
    ('', '@country'),
    ("(x,y)", "($x, $y)")
]

# Create a blank figure with labels
p = figure(plot_width = 800, plot_height = 600, 
           title = 'Deaths in Latam',
           x_axis_label = 'Days from 3 deaths', 
           y_axis_label = 'Deaths count',
           tools=select_tools)

## en el checkpoint dibujar solo dos series

for country in colormap.keys():
    country_mask =  data_ej4['country'] == country    
    country_data = data_ej4.loc[country_mask]
    
    # Add line glyph
    p.line('delta', 'coord_y', line_width = 2,
           color = colormap[country], alpha=0.8, 
           legend_label = country, source = country_data)
    
    # Add circle glyph
    p.circle('delta', 'coord_y', size = 6,
        color = colormap[country], 
        legend_label = country, source = country_data)
    
    
p.legend.location = "top_left"
p.legend.click_policy='hide'

p.add_tools(HoverTool(tooltips=tooltips))


# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

## Ejercicio 5

Repitamos el ejercicio 4 usando escala logarítmica

En este ejercicio sólo tenemos que cambiar el valor del argumento `y_axis_type` del método `figure`

In [ ]:

#Specify the selection tools to be made available
select_tools = ['box_zoom', 'pan', 'wheel_zoom', 'reset', 'crosshair', 'save']

# Format the tooltip
tooltips = [
    ('', '@country'),
    ("(x,y)", "($x, $y)")
]

# Create a blank figure with labels
p = figure(y_axis_type="log",
           plot_width = 800, plot_height = 600, 
           title = 'Deaths in Latam',
           x_axis_label = 'Days from 3 deaths', 
           y_axis_label = 'Deaths count',
           tools=select_tools)

## en el checkpoint dibujar solo dos series

for country in colormap.keys():
    country_mask =  data_ej4['country'] == country    
    country_data = data_ej4.loc[country_mask]
    
    # Add line glyph
    p.line('delta', 'coord_y', line_width = 2,
           color = colormap[country], alpha=0.8, 
           legend_label = country, source = country_data)
    
    # Add circle glyph
    p.circle('delta', 'coord_y', size = 6,
        color = colormap[country], 
        legend_label = country, source = country_data)
    
    
p.legend.location = "top_right"
p.legend.click_policy='hide'

p.add_tools(HoverTool(tooltips=tooltips))


# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

## Ejercicio 6 - Opcional

Repitamos algunos de los análisis con los datos de confirmados o recuperados en latinoamérica

## Ejercicio 7 - Opcional

Usando gráficos de barras como los que vemos acá

https://hectoramirez.github.io/covid/COVID19.html
    
analicemos los datos de latam.

---

## Plotly

Recuerden crear un usuario y generar una api key en https://chart-studio.plotly.com/

In [ ]:
import plotly
import plotly.express as px
import chart_studio

In [ ]:
#chart-studio api
username = 'completar' # your username
api_key = 'completar' # your api api_key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
import chart_studio.plotly as py

## Ejercicio 1

Vamos construir un único DataFrame con los datos de confirmados y muertos por país y fecha

Las columnas serán:
* Country/Region    
* fecha
* value_deaths
* value_confirmed




In [ ]:
def melt_data(raw_data):

    # armamos una lista con las columnas que queremos exluir
    exclude_columns = ['Province/State', 'Country/Region', 'Lat', 'Long']

    id_vars = ['Country/Region']
    value_vars = [element for element in raw_data.columns if element not in exclude_columns]
    result = pd.melt(raw_data, id_vars=id_vars, value_vars=value_vars)

    result['variable_date'] = pd.to_datetime(result['variable'])

    result.index = [result ['Country/Region'], result["variable_date"] ]

    return result


In [ ]:
world_deaths_latam_plot = melt_data(world_deaths_latam)
world_deaths_latam_plot.head(3)

In [ ]:
world_confirmed_latam_plot = melt_data(world_confirmed_latam)
world_confirmed_latam_plot.head(3)

In [ ]:
data_left = pd.DataFrame(world_confirmed_latam_plot['value'])
data_right = pd.DataFrame(world_deaths_latam_plot['value'])

data_to_plot = data_left.join(data_right, lsuffix='_confirmed', rsuffix='_deaths')

data_to_plot = data_to_plot.reset_index()

data_to_plot.head(3)

### Ejercicio 2

Vamos a graficar Confirmados vs. Muertos por país usando los datos de latam en el DataFrame que construimos en el ejercicio anterior para la máxima fecha disponible en el dataset.


In [ ]:
max_fecha = data_to_plot.variable_date.max()
max_fecha

In [ ]:
fecha_mask = data_to_plot.variable_date == max_fecha
data_to_plot_fecha = data_to_plot.loc[fecha_mask, :]
data_to_plot_fecha.head(3)

In [ ]:
fig = px.scatter(data_frame=data_to_plot_fecha, 
                 x='value_confirmed', y='value_deaths', 
                 color='Country/Region', size='value_confirmed', height=500, width=900,                 
                 text='Country/Region', title= f'Muertos vs Confirmados - {max_fecha} - (latam)')

fig.update_traces(textposition='top center')

fig.layout.update(showlegend = False)

if api_key: py.plot(fig, filename = 'scatter_muertos_confirmados', auto_open=False)
fig.show()


### Ejercicio 3

Vamos a repetir el ejercicio 2, pero en lugar de graficar la fecha máxima del dataset usaremos una animación para ver un gráfico distinto por cada fecha.


In [ ]:
min_date = data_to_plot.variable_date.min()

data_to_plot["delta_days"] = [x.days for x in (data_to_plot.variable_date - min_date)]

data_to_plot.head(3)

La cuenta gratuita no nos deja graficar tantos datos, así que vamos a gaficar sólo la última semana

In [ ]:
delta_threshold = data_to_plot.delta_days.max() - 7
data_to_plot_subset_mask = data_to_plot.delta_days >= delta_threshold
data_to_plot_subset = data_to_plot.loc[data_to_plot_subset_mask, :]
data_to_plot_subset.shape

In [ ]:
fig = px.scatter(data_frame=data_to_plot_subset, 
                 x='value_confirmed', y='value_deaths', 
                 color='Country/Region', size='value_confirmed', height=500, width=900,
                 animation_frame="delta_days",
                 text='Country/Region', title= 'Muertos vs Confirmados (latam)')

fig.update_traces(textposition='top center')

fig.layout.update(showlegend = False)

if api_key: py.plot(fig, filename = 'scatter_muertos_confirmados_animation', auto_open=False)
fig.show()


### Ejercicio 4

Ahora repitamos el ejercicio 3 para escala logarítmica

In [ ]:
fig = px.scatter(data_frame=data_to_plot_subset, 
                 x='value_confirmed', y='value_deaths', 
                 log_x=True, log_y=True,
                 color='Country/Region', size='value_confirmed', height=500, width=900,
                 animation_frame="delta_days",
                 text='Country/Region', title= 'Muertos vs Confirmados (latam) - escala logaritmica')

fig.update_traces(textposition='top center')

fig.layout.update(showlegend = False)

if api_key: py.plot(fig, filename = 'scatter_muertos_confirmados_animation', auto_open=False)
fig.show()

## Ejercicio 5

Usemos un gráfico de lineas para mostrar el total de casos confirmados de COVID 19 por pais de latam vs. cant de días desde el primer confirmado, en escala logaritmica.


In [ ]:
# borramos los indices que usamos para el join:

world_confirmed_latam_plot = world_confirmed_latam_plot.reset_index(drop = True)


confirmed_gt0_mask = world_confirmed_latam_plot.value > 0
confirmed_gt0 = world_confirmed_latam_plot.loc[confirmed_gt0_mask, :]
confirmed_gt0
min_date = confirmed_gt0.variable_date.min()
print(min_date)
#confirmed_gt0

world_confirmed_latam_plot["delta_days"] = [x.days for x in (world_confirmed_latam_plot.variable_date - min_date)]

world_confirmed_latam_plot_mask = world_confirmed_latam_plot.delta_days >= 0


world_confirmed_latam_plot = world_confirmed_latam_plot.loc[world_confirmed_latam_plot_mask, :]

world_confirmed_latam_plot.head(3)


In [ ]:
# paleta:
#https://plotly.com/python/discrete-color/

fig = px.line(world_confirmed_latam_plot, x='delta_days' , y='value', 
              log_y=True, color='Country/Region',
              color_discrete_sequence=px.colors.qualitative.Dark24,
              title='Total Casos Confirmados de COVID 19 por Pais (latam) ')

paises = world_confirmed_latam_plot["Country/Region"]


fig.layout.update(showlegend=True, 
                  yaxis =  {"title": {"text": "Numero de Personas"}},
                  xaxis =  {"title": {"text": "Cantidad de días desde el primer caso en latam"}}
                 ) 

# grabar grafica en chart-studio
if api_key: py.plot(fig, filename = 'total_casos_latam', auto_open=False)
fig.show()

## Referencias

https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-one-getting-started-a11655a467d4

https://towardsdatascience.com/your-live-covid-19-tracker-with-airflow-and-github-pages-658c3e048304

https://hectoramirez.github.io/covid/COVID19.html

https://joserzapata.github.io/es/post/covid19-visualizacion/

https://colab.research.google.com/github/JoseRZapata/JoseRZapata.github.io/blob/master/Jupyter_Notebook/Covid19_Visualizacion_es.ipynb
